In [30]:
#import stuff 
import wordle_words as wordle
from collections import Counter

In [57]:
# wordle wordlist: 
known_letters = {
    0: '',
    1: 'a',
    2: '',
    3: '',
    4: '',
}
yellow_letters =  {
    0: '',
    1: '',
    2: '',
    3: '',
    4: '',
}
wrong_letters = "cestimj"


In [58]:
possible_words = wordle.sc_words

# start by narrowing based off of known letters 
for ind, val in known_letters.items():
    if val == '' : 
        continue 
    print(f"narrowing based off of '{val}' in spot {ind}")
    possible_words = [word for word in  possible_words if word[ind] == val]
    print(f"\t list size is {len(list(possible_words))}")

# then look at all yellow letters 
for ind, val in yellow_letters.items():
    if val == '' : 
        continue 
    print(f"narrowing based off of '{val}' in spot {ind}")
    possible_words = [word for word in  possible_words if val in word and word[ind] != val]
    print(f"\t list size is {len(list(possible_words))}")
# finally look at all letters not in the words 

print("narrowing words based off of incorrect letters")
for let in wrong_letters: 
#     print(f"narrowing based off of incorrect letter '{let}'")
    possible_words = [word for word in  possible_words if let not in word ]
#     print(f'list is {possible_words}')
print(f"\t list size is {len(list(possible_words))}")


print(f"all possible words are: {list(possible_words)}")

narrowing based off of 'a' in spot 1
	 list size is 304
narrowing words based off of incorrect letters
	 list size is 42
all possible words are: ['badly', 'baggy', 'banal', 'baron', 'bawdy', 'bayou', 'daddy', 'dally', 'dandy', 'fanny', 'fauna', 'favor', 'gaudy', 'gawky', 'gayly', 'handy', 'happy', 'hardy', 'harpy', 'harry', 'kappa', 'kayak', 'labor', 'lanky', 'larva', 'laugh', 'nanny', 'naval', 'paddy', 'pagan', 'papal', 'parka', 'parry', 'radar', 'rally', 'ralph', 'randy', 'rayon', 'razor', 'valor', 'vapor', 'wagon']


In [59]:
# hard mode based geussing
hard_mode = True # currently non-hard mode is not supported  

possible_guesses = possible_words  if hard_mode else wordle.nltk_words
best_num_uniques = 0 
best_guesses = []
for guess in possible_guesses:
#     print(f'guess is {guess}')
    resps = []
    for ans in possible_words: 
        resp = ['g' if ans[i] == c else 'y' if c in ans else 'x' for i,c in enumerate(guess)]
#         print(f"word is {ans}, ans is '{''.join(resp)}'")

        resps.append(''.join(resp))
    num_uniques = len(Counter(resps))
    if num_uniques > best_num_uniques: 
        best_guesses = []
        best_num_uniques = num_uniques
    if num_uniques >= best_num_uniques: 
        entr = {
            "guess":        guess,
            "num_uniques":  num_uniques, 
            "resp_distro":  Counter(resps)
        }
        best_guesses.append(entr)
    
for guess in best_guesses:
    printafterline=False
    if guess['guess'] in possible_words and not hard_mode:
        printafterline = True 
        print("Reccomended word: ")
    print(guess)
    if printafterline : 
        print(" ")
        
    

{'guess': 'porgy', 'num_uniques': 25, 'resp_distro': Counter({'xxxxg': 9, 'xyyxx': 4, 'xxxxx': 3, 'xxxyg': 3, 'xxgxg': 2, 'xxyxg': 2, 'xxxgg': 1, 'xygxx': 1, 'xyxxy': 1, 'yxxxg': 1, 'yxgxg': 1, 'yxxxx': 1, 'xxxxy': 1, 'xxgxx': 1, 'xxxgx': 1, 'gxxxg': 1, 'gxxyx': 1, 'gxxxx': 1, 'gxgxx': 1, 'gxgxg': 1, 'xxyxx': 1, 'yxyxx': 1, 'xyyxy': 1, 'yyyxx': 1, 'xyxyx': 1})}
{'guess': 'pylar', 'num_uniques': 25, 'resp_distro': Counter({'xyxyx': 10, 'xyxyy': 4, 'xyyyx': 3, 'xxygx': 2, 'xxxyx': 2, 'xxxyg': 2, 'xxxyy': 1, 'xygyx': 1, 'yyxyx': 1, 'yyxyy': 1, 'yxxyx': 1, 'xyxgx': 1, 'xxyyg': 1, 'xxyyy': 1, 'xxyyx': 1, 'gyxyx': 1, 'gxxgx': 1, 'gxygx': 1, 'gxxyy': 1, 'gyxyy': 1, 'xxxgg': 1, 'xygyy': 1, 'yxgyy': 1, 'xxgyg': 1, 'yxxyg': 1})}


In [11]:
print(len(wordle.words))
print(len(wordle.nltk_words))
print(len(wordle.sc_words))

12972
8689
2315
